In [ ]:
# Header starts here.
from sympy.physics.units import *
from sympy import *

# Rounding:
import decimal
from decimal import Decimal as DX
from copy import deepcopy
def iso_round(obj, pv, rounding=decimal.ROUND_HALF_EVEN):
    import sympy
    """
    Rounding acc. to DIN EN ISO 80000-1:2013-08
    place value = Rundestellenwert
    """
    assert pv in set([
        # place value   #  round to:
        1,              #  1
        0.1,            #  1st digit after decimal
        0.01,           #  2nd
        0.001,          #  3rd
        0.0001,         #  4th
        0.00001,        #  5th
        0.000001,       #  6th
        0.0000001,      #  7th
        0.00000001,     #  8th
        0.000000001,    #  9th
        0.0000000001,   # 10th
        ])
    objc = deepcopy(obj)
    try:
        tmp = DX(str(float(objc)))
        objc = tmp.quantize(DX(str(pv)), rounding=rounding)
    except:
        for i in range(len(objc)):
            tmp = DX(str(float(objc[i])))
            objc[i] = tmp.quantize(DX(str(pv)), rounding=rounding)
    return objc

# LateX:
kwargs = {}
kwargs["mat_str"] = "bmatrix"
kwargs["mat_delim"] = ""
# kwargs["symbol_names"] = {FB: "F^{\mathsf B}", }

# Units:
(k, M, G ) = ( 10**3, 10**6, 10**9 )
(mm, cm) = ( m/1000, m/100 )
Newton = kg*m/s**2
Pa     = Newton/m**2
MPa    = M*Pa
GPa    = G*Pa
kN     = k*Newton
deg    = pi/180

half = S(1)/2

# Header ends here.
#
# https://colab.research.google.com/github/kassbohm/wb-snippets/blob/master/ipynb/HTM_03/Selbst/4.2_cc.ipynb

# 1:
mu, l = var("mu, l")

(l1, l2, l3) = (2*l, l, 2*l/3)
(m1, m2, m3) = (mu*l1, mu*l2, mu*l3)
(xi1, xi2, xi3) = (m1*l1**2/12, m2*l2**2/12, m3*l3**2/12)

# 2:
IC1 = xi1 * Matrix([
    [1,0,0],
    [0,0,0],
    [0,0,1],
    ])

IC2 = xi2 * Matrix([
    [0,0,0],
    [0,1,0],
    [0,0,1],
    ])

IC3 = xi3 * Matrix([
    [1,0,0],
    [0,1,0],
    [0,0,0],
    ])

pprint("\nIC1, IC2, IC3 / ( μl³ ):")
for x in [IC1, IC2, IC3]:
    tmp = x / (mu*l**3)
    pprint(tmp)

# 3:
M, dx, dy, dz = var("M, dx, dy, dz")

(dx1, dy1, dz1) = (0, l1/2, 0)
(dx2, dy2, dz2) = (l2/2, l1, 0)
(dx3, dy3, dz3) = (l2, l1, l3/2)

tmp = Matrix([
    [dy**2 + dz**2,0,0],
    [0,dx**2 + dz**2,0],
    [0,0,dx**2 + dy**2],
    ])
tmp+= Matrix([
    [     0,-dx*dy,-dx*dz],
    [-dy*dx,     0,-dy*dz],
    [-dz*dx,-dz*dy,     0],
    ])
tmp*=M

# 4:
pprint("\nI01, I02, I03 / ( μl³ ):")
IO1 = IC1 + tmp.subs([(M,m1), (dx, dx1), (dy, dy1), (dz, dz1)])
IO2 = IC2 + tmp.subs([(M,m2), (dx, dx2), (dy, dy2), (dz, dz2)])
IO3 = IC3 + tmp.subs([(M,m3), (dx, dx3), (dy, dy3), (dz, dz3)])
for x in [IO1, IO2, IO3]:
    tmp = x / (mu*l**3)
    pprint(tmp)

# 5:
pprint("\nIO / ( μl³ ):")
IO = IO1 + IO2 + IO3
IO /= mu*l**3
pprint(IO)

# 6:
pprint("\nEigenvalues of IO / (kg m²):")
ev = IO.eigenvals()
for v in ev:
    # take real part only:
    tmp = re(v)
    # multiply with μl³:
    tmp *= 4*kg/meter * (4*meter/10)**3
    tmp /= kg *meter*meter
    tmp = iso_round(tmp,0.001)
    pprint(tmp)

pprint("\nb:")
b = Matrix([1, 2, S(2)/3])
b /= b.norm()
pprint(b)

pprint("\nIOb / ( μl³ ):")
tmp = IO*b
pprint(tmp)
pprint("\nJ / ( μl³ ):")
J = b.transpose()*IO*b
J = J[0]
pprint(J)
pprint("\nJ / (kg m²):")
tmp = J
# multiply with μl³:
tmp *= 4*kg/meter * (4*meter/10)**3
tmp /= kg * meter*meter
tmp = iso_round(tmp,0.0001)
pprint(tmp)

# IC1, IC2, IC3 / ( μl³ ):
# ⎡2/3  0   0 ⎤
# ⎢           ⎥
# ⎢ 0   0   0 ⎥
# ⎢           ⎥
# ⎣ 0   0  2/3⎦
# ⎡0   0     0  ⎤
# ⎢             ⎥
# ⎢0  1/12   0  ⎥
# ⎢             ⎥
# ⎣0   0    1/12⎦
# ⎡2/81   0    0⎤
# ⎢             ⎥
# ⎢ 0    2/81  0⎥
# ⎢             ⎥
# ⎣ 0     0    0⎦
#
# I01, I02, I03 / ( μl³ ):
# ⎡8/3  0   0 ⎤
# ⎢           ⎥
# ⎢ 0   0   0 ⎥
# ⎢           ⎥
# ⎣ 0   0  8/3⎦
# ⎡4   -1    0  ⎤
# ⎢             ⎥
# ⎢-1  1/3   0  ⎥
# ⎢             ⎥
# ⎣0    0   13/3⎦
# ⎡224             ⎤
# ⎢───   -4/3  -2/9⎥
# ⎢ 81             ⎥
# ⎢                ⎥
# ⎢       62       ⎥
# ⎢-4/3   ──   -4/9⎥
# ⎢       81       ⎥
# ⎢                ⎥
# ⎣-2/9  -4/9  10/3⎦
#
# IO / ( μl³ ):
# ⎡764             ⎤
# ⎢───   -7/3  -2/9⎥
# ⎢ 81             ⎥
# ⎢                ⎥
# ⎢       89       ⎥
# ⎢-7/3   ──   -4/9⎥
# ⎢       81       ⎥
# ⎢                ⎥
# ⎣-2/9  -4/9  31/3⎦
#
# Eigenvalues of IO / (kg m²):
# 0.119
# 2.659
# 2.563
#
# b:
# ⎡3/7⎤
# ⎢   ⎥
# ⎢6/7⎥
# ⎢   ⎥
# ⎣2/7⎦
#
# IOb / ( μl³ ):
# ⎡ 374 ⎤
# ⎢ ─── ⎥
# ⎢ 189 ⎥
# ⎢     ⎥
# ⎢-5/27⎥
# ⎢     ⎥
# ⎢ 52  ⎥
# ⎢ ──  ⎥
# ⎣ 21  ⎦
#
# J / ( μl³ ):
# 88
# ──
# 63
#
# J / (kg m²):
# 0.3576
